<a href="https://colab.research.google.com/github/MeredithChou/Reinforcement/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gym

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gym
import gymnasium
import numpy as np
import time
from IPython.display import clear_output
import matplotlib.pyplot as plt

env = gym.make("Taxi-v3", render_mode = "rgb_array")
state= env.reset()

print("Initial state:", state)
print("State space: ", env.observation_space)
print("Action space: ", env.action_space)

DOWN = 0
UP = 1
RIGHT = 2
LEFT = 3
PICKUP = 4
DROPOFF = 5


#There are 500 discrete states since there are 25 taxi positions, 5 possible locations of the passenger (including the case when the passenger is in the taxi), and 4 destination locations.

Initial state: 414
State space:  Discrete(500)
Action space:  Discrete(6)


In [ ]:
def train_q(agent, env, n_episodes, max_steps=10000):
    step = 0
    steps = np.zeros(n_episodes) # Steps after each episode
    total_rewards = np.zeros(n_episodes)
    for i in range(n_episodes):
        rewards=0
        terminated = False
        truncated = False
        state = env.reset()
        while not terminated and not truncated:
            action = agent.act(state)
            state_next, reward, terminated, truncated = env.step(action)
            agent.learn(state, action, reward, state_next,i)
            state = state_next
            step += 1
            rewards += reward
            
            if step>max_steps:
                return steps, rewards
        steps[i] = step
        total_rewards[i] = rewards
        
    print(agent.Q[422,:])
    print(agent.Q[8,:])
    print(agent.Q[209,:])
    print(agent.Q[227,:])
    print(agent.Q[386,:])
    return total_rewards, steps

In [ ]:
class QAgent():
    def __init__(self, n_states, n_actions, gamma, alpha, epsilon):
        self.n_states = n_states
        self.n_actions = n_actions
        self.alpha = 0.1
        self.gamma = 1
        self.epsilon = 0
        self.Q = np.zeros((n_states, n_actions))
        self.Qtables = np.zeros((n_states, n_actions+1))
        
        for i in range(0,500):
            self.Qtables[i,0] = i
        
        
    def act(self, state):
        # Implement the self.epsilon-greedy policy
        rand = np.random.rand()
#         if np.random.rand() > self.epsilon: 
        if rand > self.epsilon: 
#             print(f"rand: {rand}, self.epsilon: {self.epsilon}")
            action = np.argmax(self.Q[state,:]) 
#             print(f"Now: rand > epsilon , This state action array: {self.Q[state,:]}, will take action: {action}")
        else: 
#             print(f"rand: {rand}, self.epsilon: {self.epsilon}")
            action = np.random.choice(self.n_actions) 
#             print(f"Now: rand < epsilon , Can selete action from {self.n_actions} ways, will take action: {action}")
            
        return action
            
    def learn(self, s, a, r, s_next,i):
        # Implement the Q-learning update
        Q_next = np.max(self.Q[s_next,:]) 
        self.Q[s,a] += self.alpha*(r + self.gamma*Q_next - self.Q[s,a]) 
        
#         if i == 50:
#             print(i,self.Q[283,:])

In [ ]:
env_train = gym.make("Taxi-v3")
agentQ = QAgent(env.observation_space.n, env.action_space.n, gamma=1, alpha=0.1, epsilon=0)

In [ ]:
result = train_q(agentQ, env_train, n_episodes=10000)

[-6.         -5.99619936 -6.12500962 -6.         -6.         -6.        ]
[-4.99997588 -5.         -5.07537815 -5.         -5.         -5.        ]
[-2.99999994 -3.00166143 -3.08228234 -3.         -3.         -3.        ]
[-6.00476157 -5.99405593 -5.95348923 -5.97511311 -6.         -6.        ]
[-4.00823301 -3.9999934  -4.         -4.12445905 -4.         -4.        ]
